In [4]:
!pip install pandarallel
import pandas as pd
from bs4 import BeautifulSoup
import requests
import glob
import numpy as np
from pandarallel import pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df=pd.DataFrame()
read_file = pd.read_csv('//content/drive/MyDrive/Colab Notebooks/laopinioncoruna/url.csv', header=[0])
df=pd.concat([df,read_file]).reset_index(drop=True)
print(len(df))

26490


In [7]:
def get_articles(url):
  try:
    r1 = requests.get(url)
    coverpage = r1.content
    soup = BeautifulSoup(coverpage, 'html.parser')
    
    try:
      coverpage=soup.find_all('div',class_='headline-article')
      title=coverpage[0].find('h1').get_text()
      try:
       description=coverpage[0].find('h2').get_text()
      except:
        description=''
    except:
      title=''
      description=''
    
    try:
      coverpage=soup.find_all('div',class_='article-entry-author')
      time=coverpage[0].find('span',class_='article-author__date').get_text()
    except:
      time=''  

    try:
      coverpage=soup.find_all('div',class_='article-body')
      article=''
      for i in coverpage[0].find_all('p'):
        article=article+i.get_text()
      # print(article)
    except:
      article=''
      print('article error\n'+url)


  except:
    title=''
    description=''
    time=''
    article=''
    print('url error\n'+url)    

  return [title,description,time,article]

In [8]:
get_articles('https://www.laopinioncoruna.es/mundo/2022/08/07/espana-pide-inmediato-alto-fuego-73155443.html')

['España pide un alto el fuego inmediato ante una nueva escalada de violencia en Gaza',
 'Este domingo, en su tercer día, la escalada de violencia deja ya 41 muertos y más de 300 heridos',
 '\n                    07·08·22\n            ',
 '\n        El Gobierno de España sigue con gran preocupación la nueva escalada de violencia en Gaza y saluda la posibilidad de un alto el fuego, que "debe aplicarse de inmediato para detener la pérdida de vidas humanas".\n\n        A través de un comunicado, España reitera su firme condena al lanzamiento de cohetes, y aunque que reconoce "el derecho de Israel a proteger a su población", también pide que "se evite a toda costa causar víctimas civiles entre la población palestina".\n\n        El Gobierno de España considera "esencial recuperar la perspectiva política para aplicar la solución de los dos estados" y cree que es necesario "continuar los esfuerzos que se han hecho recientemente para crear una situación más sostenible en Gaza".\n\n        La 

In [10]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)

output=df['rows'].parallel_apply(get_articles)

df['country']='La Coruna'
df['title']=''
df['description']=''
df['time']=''
df['article']=''

for i in range(0,len(df)):
  df.loc[i]['title','description','time','article']=output[i]

df = df[df.title.str.len()!=0].reset_index(drop=True)
df = df.dropna()
str_name='/content/drive/MyDrive/Colab Notebooks/laopinioncoruna/url_article.csv'
print(df)
df.to_csv(str_name, index=False)

INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


article error
https://www.laopinioncoruna.es/mundo/2021/08/06/cuatro-heridos-ataque-cuchillo-tren-55991699.html
article error
https://www.laopinioncoruna.es/mundo/2022/05/31/conflicto-ucrania-rusia-directo-63011276.html
article error
https://www.laopinioncoruna.es/mundo/2021/09/03/mujeres-afganas-toman-calles-kabul-56912943.html
article error
https://www.laopinioncoruna.es/mundo/2022/03/08/evacuacion-civiles-ciudades-ucrania-63593475.html
article error
https://www.laopinioncoruna.es/mundo/2022/05/03/espana-busca-involucrar-otan-flanco-65657407.html
article error
https://www.laopinioncoruna.es/mundo/2022/03/31/conflicto-ucrania-rusia-directo-63011276.html
article error
https://www.laopinioncoruna.es/espana/2021/10/04/dos-migrantes-fallecidos-cabrera-desaparecidos-58003900.html
article error
https://www.laopinioncoruna.es/espana/2021/04/06/sanchez-estrena-foto-familia-gobierno-46113864.html
article error
https://www.laopinioncoruna.es/espana/2021/09/29/sido-llegada-lava-volcan-palma-5780